# Inference with classifiers

# Parameter estimation: 2D Gaussian example

In [ ]:
# If running on Google Colab: install dependencies
!pip install numpy pandas matplotlib scikit-learn torch --quiet

This notebook will describe how to perform parameter estimation (with the extraction of confidence intervals) using ML classifiers. Note, this is essentially an extension of the hypothesis testing procedure where the alternative hypothesis $\mathcal{H}_1$ is a "composite" hypothesis, containing the ensemble of all possible values for the parameter of interest. As discussed in the introduction, for parameter estimation we need to learn the conditional probability density: $p(x|\theta)$.
This quantity tells us the probabilty of observing data $x$ given parameter value $\theta$. In fact, what we want to learn for inference with classifiers is the conditional density ratio:

$$
\frac{p(x|\theta)}{p(x|\theta_0)}
$$

where $\theta_0$ is some reference value of the parameter. We can then define a log-likelihood-ratio test-statistic to infer $\theta$ from the data, and its corresponding confidence intervals. All we need is a faithful simulator that can generate $x$ for any value of $\theta$.

The parameter estimation with SBI discussed in this notebook is founded on the concept of parametric classifiers [1]. These models are trained such that the classifier output is "parametric" in $\theta$: $\hat{f}(x|\theta)$ i.e. the decision function depends on the value of $\theta$. 

In practice, we learn this by simply adding $\theta$ as an additional parameter to the model training:

![parametric neural network](pnn.png)

Crucially, one must initially make the distribution of the conditional feature ($\theta$) the same between the two classes. 

Let's go through the training steps in detail...

1) Class 0: generate $N$ samples using the simulator for the reference hypothesis $\theta = \theta_0$ i.e. draw samples from $p(x|\theta_0)$ for fixed $\theta_0$. It is up to you what to choose for $\theta_0$ i.e. we must pick a choice for the reference hypothesis. In theory, the final parameter estimation will be independent of this choice, but it helps to pick something sensible which is not too far from the observed parameter values.

2) Class 1: generate $N$ samples using the simulator with different values of $\theta$ i.e. draw samples from $p(x|\theta)$ for various $\theta$. If possible, one should generate the samples to be continuous in the values of $\theta$ i.e. randomly sampled between some upper and lower ranges, where the ranges are chosen to be sufficiently wide for the confidence interval level that you want to probe. In practice it is often much easier to generate subsamples with discrete steps in $\theta$. The method still works as long as the discrete steps are sufficiently fine-grained (compared to the sensitivity). We will use the latter approach in this notebook.

<div style="background-color:#FFCCCB">

3) Consider training a classifier $\hat{f}(x,\theta)$ to distinguish between Class 0 and Class 1 with $\{x,\theta\}$ as input features. After the likelihood-ratio trick, we arrive at an approximation of the joint-likelihood ratio between the two classes:
    $$
    \frac{\hat{f}(x_i,\theta)}{1-\hat{f}(x_i,\theta)} \approx \frac{p(x_i,\theta|\mathrm{Class\,1})}{p(x_i,\theta|\mathrm{Class\,0})}
    $$
    If we expand the joint distributions we obtain:
    $$
    \frac{p(x_i,\theta|\mathrm{Class\,1})}{p(x_i,\theta|\mathrm{Class\,0})} = \frac{p(x_i|\theta)p(\theta|\mathrm{Class\,1})}{p(x_i|\theta_0)p(\theta|\mathrm{Class\,0})}
    $$
    What we want is only the conditional density ratio:
    $$
    \frac{p(x_i|\theta)}{p(x_i|\theta_0)}
    $$
    Hence, we need to make sure the conditional parameter distributions are the same between the two classes:
    $$
    p(\theta|\mathrm{Class\,1}) = p(\theta|\mathrm{Class\,0})
    $$
    so that the terms cancel in the joint density ratio, and we are left with just the conditional density ratio. 
    
    In other words, we need the $\theta$ parameter distributions in Class 0 and Class 1 to have identical densities. In practice, this means using the same sampling distribution for $\theta$ in both classes. If they differ, the classifier will pick up class-specific information about how often certain $\theta$ values appear in each class, rather than purely how likely the data $x$ are given $\theta$. This would break the interpretation of the classifier output as a conditional density ratio.

    So before training we need to artificially generate a $\theta$ distribution for Class 0 by sampling from the Class 1 distribution. 

</div> 

4) After the $\theta$ distributions between the two classes are made to be the same in training, the classifier output can be used to approximate the conditional density ratio. We will show later on how this can be used for parameter estimation.

[1] - P. Baldi et al., *Parameterized neural networks for high-energy physics*, Eur. Phys. J. C 76 (2016) 235. [arXiv:1601.07913](https://www.arxiv.org/abs/1601.07913)

## 2D Gaussian example

In this notebook we will follow a simple example, where the analytic likelihood is known. The example is described below:

### Dataset:

Two observables ($x_1, x_2$) which follow a 2D Gaussian distribution. The mean and width related to $x_1$, $\mu_1$ and $\sigma_1$, are known. Only the width related to $x_2$, $\sigma_2$, is known. The two unknowns are $\mu_2$ and the correlation between the observables $\rho_{12}$. We are provided with a dataset of $N_{obs}=20$ samples of ($x_1, x_2$) values stored in `data_parameter_estimation.csv`. 

### Aim:

Infer the values of $\mu_2$ and $\rho_{12}$ from the data, along with the corresponding confidence intervals. Comparing to the notation above $x = \{x_1,x_2\}$ and $\theta = \{\mu_2,\rho_{12}\}$.

### Simulation 

We have a simulator that can generate samples from a 2D Gaussian with arbitrary $\mu_1$, $\sigma_1$, $\mu_2$, $\sigma_2$, $\rho_{12}$. The simulation can generate $N$ samples using the following code:
```python
sim = run_simulation(N, mu1, sigma1, mu2, sigma2, rho12)
```

### Analytic solution

<div style="background-color:#FFCCCB">

The analytic (conditional) probability density for a 2D Gaussian is:

$$
p(x_1,x_2|\mu_1,\sigma_1,\mu_2,\sigma_2,\rho_{12})
=
\frac{1}{2\pi\,\sigma_1\sigma_2\sqrt{1-\rho_{12}^2}}
\exp\!\left(
-\frac{1}{2(1-\rho_{12}^2)}
\left[
\frac{(x_1-\mu_1)^2}{\sigma_1^2}
+
\frac{(x_2-\mu_2)^2}{\sigma_2^2}
-
\frac{2\rho_{12}(x_1-\mu_1)(x_2-\mu_2)}{\sigma_1\sigma_2}
\right]
\right).
$$

We will use this to compare to the "learned" conditional density ratio.

</div>

### Tasks

1. Run the simulation and compare to the observed data
1. Extract analytic result
1. Train a parametric classifier
1. Use parametric classifier output to estimate the values and confidence intervals of $\mu_2$ and $\rho_{12}$.

## Run simulation and data exploration
We will begin by writing a python function to run our simulation.

In [31]:
def run_simulation(N, mu1=0, sigma1=1, mu2=0, sigma2=1, rho12=0):
    cov_matrix = [[sigma1**2, rho12 * sigma1 * sigma2],
                  [rho12 * sigma1 * sigma2, sigma2**2]]
    data = np.random.multivariate_normal(mean=[mu1, mu2], cov=cov_matrix, size=N)
    df = pd.DataFrame(data, columns=['x1', 'x2'])
    return df

We first pick a sensible reference hypothesis ($\theta=\theta_0$). For this example, we will pick a 2D unit Gaussian with zero correlation.

Let's run the simulation for our reference sample ($\mathcal{H}_0$) and for a potential alternative hypothesis (from the ensemble of alternative hypotheses). In this example we know $\mu_1=0, \sigma_1=1$ and $\sigma_2=1$, so we only vary $\mu_2$ and $\rho_{12}$ in the alternative hypothesis.

In [32]:
# Generate synthetic training data
num_train_per_class = 100000

# H0: reference sample
sim_H0 = run_simulation(num_train_per_class, mu1=0, sigma1=1, mu2=0, sigma2=1, rho12=0)

# H1: example alternative hypothesis
sim_H1 = run_simulation(num_train_per_class, mu1=0, sigma1=1, mu2=0.5, sigma2=1, rho12=0.5)

# Load the data from csv file
data_obs = pd.read_csv('data_parameter_estimation.csv')
N_obs = len(data_obs)

In [ ]:
# Plot 2D heatmaps for (x1,x2) and overlay observed data
# Show for both reference sample and example alternative hypothesis
fig, axs = plt.subplots(1, 2, figsize=(10,4))
axs[0].hist2d(sim_H0['x1'], sim_H0['x2'], bins=50, range=[[-4,4],[-4,4]], density=True, cmap='Blues')
axs[0].scatter(data_obs['x1'], data_obs['x2'], color='black', label='Observed Data')
axs[0].set_xlabel('x1')
axs[0].set_ylabel('x2')
axs[0].set_title('H0: reference')
axs[0].legend()
# Add text box with parameter values for reference sample
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
textstr = '\n'.join((
    r'$\mu_1=0$ (known)',
    r'$\sigma_1=1$ (known)',
    r'$\mu_2=0$',
    r'$\sigma_2=1$ (known)',
    r'$\rho_{12}=0$'))
axs[0].text(0.05, 0.95, textstr, transform=axs[0].transAxes, fontsize=8,
        verticalalignment='top', bbox=props)

axs[1].hist2d(sim_H1['x1'], sim_H1['x2'], bins=50, range=[[-4,4],[-4,4]], density=True, cmap='Oranges')
axs[1].scatter(data_obs['x1'], data_obs['x2'], color='black', label='Observed Data')
axs[1].set_xlabel('x1')
axs[1].set_ylabel('x2')
axs[1].set_title('H1: example alternative hypothesis')
axs[1].legend()
# Add text box with parameter values for example alternative hypothesis
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
textstr = '\n'.join((
    r'$\mu_1=0$ (known)',
    r'$\sigma_1=1$ (known)',
    r'$\mu_2=0.5$',
    r'$\sigma_2=1$ (known)',
    r'$\rho_{12}=0.5$'))
axs[1].text(0.05, 0.95, textstr, transform=axs[1].transAxes, fontsize=8,
        verticalalignment='top', bbox=props)
plt.show()

Does the data look more like the reference hypothesis or the example alternative hypothesis? It is hard to tell. Let's perform a proper parameter estimation.

## Analytic calculation of test statistic
We will use the analytic form of the density to calculate the log-likelihood ratio. With this formula, we can calculate the test-statistic ($2\Delta\mathrm{NLL}$) as a function of $(\mu_2,\rho_{12})$ for the observed data. From this we can determine the best-fit value (minimum) and the corresponding confidence intervals.

Make sure you understand the form of the function below...

In [34]:
def log_likelihood_ratio_analytic(x, params, params_ref):
    mu1, sigma1, mu2, sigma2, rho12 = params
    mu1_ref, sigma1_ref, mu2_ref, sigma2_ref, rho12_ref = params_ref

    cov_matrix = np.array([[sigma1**2, rho12 * sigma1 * sigma2],
                           [rho12 * sigma1 * sigma2, sigma2**2]])
    cov_matrix_ref = np.array([[sigma1_ref**2, rho12_ref * sigma1_ref * sigma2_ref],
                               [rho12_ref * sigma1_ref * sigma2_ref, sigma2_ref**2]])
    
    inv_cov = np.linalg.inv(cov_matrix)
    inv_cov_ref = np.linalg.inv(cov_matrix_ref)
    
    diff = x - np.array([mu1, mu2])
    diff_ref = x - np.array([mu1_ref, mu2_ref])

    ll = -0.5 * np.einsum('ij,jk,ik->i', diff, inv_cov, diff) - 0.5 * np.log(np.linalg.det(cov_matrix)) - np.log(2 * np.pi)
    ll_ref = -0.5 * np.einsum('ij,jk,ik->i', diff_ref, inv_cov_ref, diff_ref) - 0.5 * np.log(np.linalg.det(cov_matrix_ref)) - np.log(2 * np.pi)
    return ll-ll_ref


In [35]:
# Scan over mu2 and rho12 values and calculate llr for observed data at each point
mu2_points = np.linspace(-1, 1, 100)
rho12_points = np.linspace(-0.99, 0.99, 100)
llr_vals = np.zeros((len(mu2_points), len(rho12_points)))
params_ref = (0, 1, 0, 1, 0)  # Reference parameters (H0)
for i, mu2 in enumerate(mu2_points):
    for j, rho12 in enumerate(rho12_points):
        params = (0, 1, mu2, 1, rho12)  # Current parameters
        llr = log_likelihood_ratio_analytic(data_obs[['x1', 'x2']].values, params, params_ref)
        llr_vals[i, j] = np.sum(llr)

# Convert llr to test statistic
test_statistic_analytic = -2 * (llr_vals-np.max(llr_vals))

We can plot the test-statistic surface $t(\mu_2,\rho_{12})$ in the $(\mu_2,\rho_{12})$ plane. The best-fit parameter value corresponds to the minimum of the test-statistic curve. 

Using Wilk's theorem [1], this $2\Delta\mathrm{NLL}$ test-statistic distribution asymptotically approaches the $\chi^2$-distribution with $n=2$ degrees of freedom. We will use this "asymptotic approximation" to estimate the 68% and 95% confidence intervals. These are the union of points in the plane for which $2\Delta\mathrm{NLL}$ is less than 2.3 and 5.99, respectively. In principle, one should check the validity of Wilk's theorem with the Neyman construction of confidence intervals (using toys), but we will not go into that in this notebook.

Let's plot the test-statistic surface, and highlight the best-fit, 68% and 95% CL intervals for the $(\mu_2,\rho_{12})$ parameters. 

[1] - Wilk's Theorem. [Wikipedia link](https://en.wikipedia.org/wiki/Wilks%27_theorem)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
# Set limit on color scale as t=20
c = ax.imshow(test_statistic_analytic.T, extent=[mu2_points[0], mu2_points[-1], rho12_points[0], rho12_points[-1]],
              origin='lower', aspect='auto', cmap='viridis', vmin=0, vmax=20)
ax.set_xlabel(r'$\mu_2$')
ax.set_ylabel(r'$\rho_{12}$')
fig.colorbar(c, ax=ax, label=r'Test Statistic $t = 2\Delta NLL$')
# Overlay contour lines
contours = ax.contour(mu2_points, rho12_points, test_statistic_analytic.T, levels=[2.30, 5.99], colors='red', linestyles='dashed')
ax.clabel(contours, inline=True, fontsize=8, fmt={2.30: '68% CL', 5.99: '95% CL'})
# Add minimum point
min_idx = np.unravel_index(np.argmin(test_statistic_analytic), test_statistic_analytic.shape)
ax.plot(mu2_points[min_idx[0]], rho12_points[min_idx[1]], marker='x', color='red', markersize=5, label='Best Fit', linestyle='None')
ax.legend()
plt.show()


The data favours a positive correlation ($\rho_{12}$), with a mean $\mu_2$ around zero.

## Training a parametric classifier
Now we will perform the parameter estimation using SBI.

As described above we need to generate samples for Class 1 i.e. where we draw samples from the simulator for various values of $\theta = \{\mu_2,\rho_{12}\}$. 

We will create subsamples with small discrete points in a grid of $\mu_2$ and $\rho_{12}$.

In [37]:
sim_H1_ensemble = []
mu2_train_vals = np.linspace(-1,1,50)
rho12_train_vals = np.linspace(-0.9999,0.9999,50) 
# Calculate the number of training samples per parameter point to keep total = num_train_per_class
num_train_per_subsample = num_train_per_class // (len(mu2_train_vals) * len(rho12_train_vals))
for mu2 in mu2_train_vals:
    for rho12 in rho12_train_vals:
        sim_subsample = run_simulation(num_train_per_subsample, mu1=0, sigma1=1, mu2=mu2, sigma2=1, rho12=rho12)
        sim_subsample['mu2'] = mu2
        sim_subsample['rho12'] = rho12
        sim_H1_ensemble.append(sim_subsample)

# Concatenate all subsamples into a single dataframe and add label
sim_H1 = pd.concat(sim_H1_ensemble, ignore_index=True)
sim_H1['label'] = 1

This next step is crucial to ensure that we learn the conditional density ratio. We need to match the distribution of the conditional parameters $\mu_2,\rho_{12}$ in Class 0 (reference) to Class 1. This ensures that the classifier will not pick up class-specific information about how often certain $\theta$ values appear in each class, and instead will learn how likely the data $x$ are given $\theta$.

To do this, for each sample in Class 0 we randomly choose $\mu_2,\rho_{12}$ values from the possible discrete values used to generate Class 1. 

In [38]:
sim_H0['mu2'] = np.random.choice(mu2_train_vals, size=len(sim_H0))
sim_H0['rho12'] = np.random.choice(rho12_train_vals, size=len(sim_H0))

# Add label for H0
sim_H0['label'] = 0

Let's explicitly check that the conditional parameter distributions are identical (give or take statistical fluctuations)...

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))
ax[0].hist(sim_H0['mu2'], bins=50, range=(-1,1), alpha=0.5, label='Class 0')
ax[0].hist(sim_H1['mu2'], bins=50, range=(-1,1), alpha=0.5, label='Class 1')
ax[0].set_xlabel(r'$\mu_2$')
ax[0].set_ylabel('Counts')
ax[0].set_ylim(0, ax[0].get_ylim()[1]*1.2)
ax[0].legend(loc='best')

ax[1].hist(sim_H0['rho12'], bins=50, range=(-1,1), alpha=0.5, label='Class 0')
ax[1].hist(sim_H1['rho12'], bins=50, range=(-1,1), alpha=0.5, label='Class 1')
ax[1].set_xlabel(r'$\rho_{12}$')
ax[1].set_ylabel('Counts')
ax[1].set_ylim(0, ax[1].get_ylim()[1]*1.2)
ax[1].legend(loc='best')

plt.show()

Excellent! Now that the conditional parameter distributions are the same, we can correctly train a parametric classifier.

We will again use a feed-forward MLP for this (relatively) simple example.

Let's first prepare the datasets for training and then build the model. Note, the model takes as input both $(x_1,x_2)$ and $(\mu_2,\rho_{12})$...

In [40]:
# Combine datasets and split into train/test sets
sim = pd.concat([sim_H0, sim_H1], ignore_index=True)
test_size = 0.2
sim_train, sim_test = train_test_split(sim, test_size=test_size, shuffle=True)

# Function to convert pandas dataframe to a torch tensor
def df_to_tensor(df, feature_cols):
    return torch.tensor(df[feature_cols].values, dtype=torch.float32)

# Convert to torch tensors
feature_cols = ['x1', 'x2', 'mu2', 'rho12']
X_train = df_to_tensor(sim_train, feature_cols)
y_train = df_to_tensor(sim_train, ['label'])
X_test = df_to_tensor(sim_test, feature_cols)
y_test = df_to_tensor(sim_test, ['label'])
# Create DataLoader for batching
batch_size = 2048
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
# Define MLP model with optional number of hidden layers and units
class ParameterizedMLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_hidden_layers=2):
        super(ParameterizedMLP, self).__init__()
        layers = [nn.Linear(input_size, hidden_size), nn.ReLU()]
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size, 1))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
# Initialize model, loss function and optimizer
input_size = 4  # x1, x2, mu2, rho12
hidden_size = 128
num_hidden_layers = 3
model = ParameterizedMLP(input_size, hidden_size, num_hidden_layers)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
train_losses = []
test_losses = []
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * inputs.size(0)
    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    model.eval()
    running_test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_test_loss += loss.item() * inputs.size(0)
    epoch_test_loss = running_test_loss / len(test_loader.dataset)
    test_losses.append(epoch_test_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Test Loss: {epoch_test_loss:.4f}')

In [ ]:
# Plot training and testing loss curves
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
ax.plot(range(1, num_epochs+1), test_losses, label='Test Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.legend()
plt.show()

## Using the parametric classifier for inference
Now that the model has been trained, we can perform the parameter estimation.

This is done by evaluating the classifier output for the observed data sample $x_i$ at fixed values of the parameters of interest: $\hat{f}(x_i|\mu_2,\rho_{12})$.

<div style="background-color:#FFCCCB">

Using the likelihood-ratio-trick, we obtain an approximation of the conditional density ratio:

$$
\frac{\hat{f}(x_i|\mu_2,\rho_{12})}{1-\hat{f}(x_i|\mu_2,\rho_{12})} \approx \frac{p(x_i|\mu_2,\rho_{12})}{p(x_i|\mu_2=0,\rho_{12}=0)}
$$

Just like in the hypothesis-testing procedure, we can use this to approximate the log-likelihood-ratio test statistic over the whole data set:

$$
2\Delta\mathrm{NLL}(\mu_2,\rho_{12}) \approx -2  \sum^{N_{obs}}_{x_i \in \mathcal{D}} \ln{\bigg(\frac{\hat{f}(x_i|\mu_2,\rho_{12})}{1-\hat{f}(x_i|\mu_2,\rho_{12})}\bigg)}
$$

where crucially the test statistic is now a function of the parameters of interest. 

</div>

We evaluate the "learned" test statistic values in a grid of $(\mu_2,\rho_{12})$ to build up the test statistic surface. We can then use this surface to infer the best-fit parameter values (minimum) and their respective confidence intervals.

The final test-statistic is shifted so that the minimum is at zero:

$$
2\mathrm{NLL}(\mu_2,\rho_{12}) - 2\mathrm{NLL}_{\mathrm{min}}
$$

Let's again write python functions to calculate the test-statistic using the classifier output:

In [44]:
def log_likelihood_ratio_parameterized(X, mu2_eval, rho12_eval, model, clip=1e-10):
    N = X.shape[0]
    mu2_array = np.full(N, mu2_eval)
    rho12_array = np.full(N, rho12_eval)
    input_array = np.column_stack((X, mu2_array, rho12_array))
    with torch.no_grad():
        scores = model(torch.tensor(input_array, dtype=torch.float32)).numpy().flatten()
    # Avoid log(0) by clipping scores
    scores = np.clip(scores, clip, 1-clip)
    llr = np.log(scores / (1 - scores))
    return llr

As before, we scan over the $(\mu_2,\rho_{12})$ plane but now calculate the learned test statistic for each point. We will then compare this to the result from the analytic solution.

In [51]:
mu2_points = np.linspace(-1, 1, 100)
rho12_points = np.linspace(-0.99, 0.99, 100)
llr_vals_clf = np.zeros((len(mu2_points), len(rho12_points)))
for i, mu2 in enumerate(mu2_points):
    for j, rho12 in enumerate(rho12_points):
        llr = log_likelihood_ratio_parameterized(data_obs[['x1', 'x2']].values, mu2, rho12, model)
        llr_vals_clf[i, j] = np.sum(llr)

# Convert llr to test statistic
test_statistic_clf = -2 * (llr_vals_clf-np.max(llr_vals_clf))

In [ ]:
# Plot test-statistic heatmap from parameterized classifier next to analytic solution
fig, ax = plt.subplots(1, 2, figsize=(14,5))
c1 = ax[0].imshow(test_statistic_analytic.T, extent=[mu2_points[0], mu2_points[-1], rho12_points[0], rho12_points[-1]],
              origin='lower', aspect='auto', cmap='viridis', vmin=0, vmax=20)
ax[0].set_xlabel(r'$\mu_2$')
ax[0].set_ylabel(r'$\rho_{12}$')
fig.colorbar(c1, ax=ax[0], label=r'Test Statistic $t = 2\Delta NLL$ (Analytic)')
contours1 = ax[0].contour(mu2_points, rho12_points, test_statistic_analytic.T, levels=[2.30, 5.99], colors='red', linestyles='dashed')
ax[0].clabel(contours1, inline=True, fontsize=8, fmt={2.30: '68% CL', 5.99: '95% CL'})
min_idx_analytic = np.unravel_index(np.argmin(test_statistic_analytic), test_statistic_analytic.shape)
ax[0].plot(mu2_points[min_idx_analytic[0]], rho12_points[min_idx_analytic[1]], marker='x', color='red', markersize=5, label='Best Fit', linestyle='None')
ax[0].legend()

c2 = ax[1].imshow(test_statistic_clf.T, extent=[mu2_points[0], mu2_points[-1], rho12_points[0], rho12_points[-1]],
                origin='lower', aspect='auto', cmap='viridis', vmin=0, vmax=20)
ax[1].set_xlabel(r'$\mu_2$')
ax[1].set_ylabel(r'$\rho_{12}$')
fig.colorbar(c2, ax=ax[1], label=r'Test Statistic $t = 2\Delta NLL$ (Classifier)')
contours2 = ax[1].contour(mu2_points, rho12_points, test_statistic_clf.T, levels=[2.30, 5.99], colors='blue', linestyles='dashed')
ax[1].clabel(contours2, inline=True, fontsize=8, fmt={2.30: '68% CL', 5.99: '95% CL'})
min_idx_clf = np.unravel_index(np.argmin(test_statistic_clf), test_statistic_clf.shape)
ax[1].plot(mu2_points[min_idx_clf[0]], rho12_points[min_idx_clf[1]], marker='x', color='blue', markersize=5, label='Best Fit', linestyle='None')
ax[1].legend()

plt.show()

We can also visualise how well we have done in learning the test statistic by taking the difference of the test statistic surfaces, or by comparing the contours on the same plot.

In [ ]:
# Plot difference of test-statistic heatmaps next to plot with all contours on same axis
fig, ax = plt.subplots(1, 2, figsize=(14,5))
diff_test_statistic = test_statistic_clf - test_statistic_analytic
c = ax[0].imshow(diff_test_statistic.T, extent=[mu2_points[0], mu2_points[-1], rho12_points[0], rho12_points[-1]],
              origin='lower', aspect='auto', cmap='BrBG', vmin=-5, vmax=5)
ax[0].set_xlabel(r'$\mu_2$')
ax[0].set_ylabel(r'$\rho_{12}$')
fig.colorbar(c, ax=ax[0], label=r'Difference in Test Statistic $t_{clf} - t_{analytic}$')
# Add contours for reference
contours_analytic = ax[0].contour(mu2_points, rho12_points, test_statistic_analytic.T, levels=[2.30, 5.99], colors='red', linestyles='dashed')
contours_clf = ax[0].contour(mu2_points, rho12_points, test_statistic_clf.T, levels=[2.30, 5.99], colors='blue', linestyles='dotted')

# Plot all contours on same axis
ax[1].set_xlabel(r'$\mu_2$')
ax[1].set_ylabel(r'$\rho_{12}$')
contours_analytic = ax[1].contour(mu2_points, rho12_points, test_statistic_analytic.T, levels=[2.30, 5.99], colors='red', linestyles='dashed')
ax[1].clabel(contours_analytic, inline=True, fontsize=8, fmt={2.30: '68% CL', 5.99: '95% CL'})
contours_clf = ax[1].contour(mu2_points, rho12_points, test_statistic_clf.T, levels=[2.30, 5.99], colors='blue', linestyles='dotted')
ax[1].clabel(contours_clf, inline=True, fontsize=8, fmt={2.30: '68% CL', 5.99: '95% CL'})
ax[1].plot(mu2_points[min_idx_analytic[0]], rho12_points[min_idx_analytic[1]], marker='x', color='red', markersize=5, label='Analytic', linestyle='None')
ax[1].plot(mu2_points[min_idx_clf[0]], rho12_points[min_idx_clf[1]], marker='x', color='blue', markersize=5, label='Classifier', linestyle='None')
ax[1].legend(loc='lower right')

plt.show()

## Summary

We have done a reasonable job at approximating the test statistic surface. This allows us to infer $\mu_2$ and $\rho_{12}$ from the observed data without a-priori knowing the likelihood! For reference, the true values used to generate the dataset were $\mu^{\mathrm{true}}_2 = -0.1$ and $\rho^{\mathrm{true}}_{12} = 0.3$. This is consistent with our SBI measurement within the 68% confidence level interval. 

That said, our SBI model is not perfect and there is plenty of room for improvement. Differences in the learned likelihood-ratio compared to the true likelihood-ratio will lead to a biased estimator, and therefore can lead to biased measurements. In a worst-case scenario, this may lead to incorrect conclusions! In Exercise 1 below, we will explore how we can improve the training procedure to provide a more accurate test-statistic prediction. Ultimately, when using SBI in real research settings, significant time is spent validating the SBI models to ensure that we do not bias our measurements.

Although this example (2D Gaussian with two unknown parameters) is overkill, we have demonstrated all the steps necessary to perform parameter estimation with a ML classifier. You can imagine how useful this becomes for more complex problems when the true likelihood is not known. We can leverage the benefits of machine learning to perform inference in high-dimensions in a complex feature space, thereby squeezing every drop of information out of the data.

<div style="background-color:#C2F5DD">

## Exercises

### 5.1 Improving the accuracy of the parameter estimation
In the example above the "learned" test statistic shows some deviations from the analytic solution. This could lead to biases (or incorrect conclusions) when using the ML SBI approach. Can you improve the accuracy by altering the model training? One could consider:
* Increasing the amount of training data
* Increasing the granularity of the $(\mu_2,\rho_{12})$ grid used for the training
* Extending to a more complex model arhcitecture

Reproduce the plots at the end of this section to see how your changes affect the accuracy.

</div>

In [ ]:
# YOUR CODE HERE

<div style="background-color:#C2F5DD">

### 5.2 Amortized inference
As before, we can use the trained parametric classifier to perform inference on new observations. Try generating data for a particular $(\mu_2,\rho_{12})$ set, starting with $N=20$ samples, and perform inference on this new dataset. Then generate $N=50$ samples and repeat the inference. How do the confidence intervals change? Is the "true" value used to generate the samples consistent with the results? How does this change for $N=100$, $N=500$ etc? Do you eventually observe a sizeable bias in the results?

</div>

In [1]:
# YOUR CODE HERE

<hr style="border:2px solid gray">

<div style="background-color:#efdff2">

# 6. Appendix: further reading <a id='further_reading'></a>

SBI is a fast-evolving field with applications across many domains in science. A nice overview of the topic can be found in the following reference:

[1] K. Cranmer, J. Brehmer, G. Louppe, *The frontier of simulation-based inference*. PNAS (2020). [arXiv:1911.01429](https://arxiv.org/pdf/1911.01429)

The following sections provide a bit of a deeper dive into some interesting topics. This is by no means an exhaustive list, but is a good place to get started on some cutting-edge research topics.

## Validating and calibrating the classifier
As we have seen in [Section 5](#parameter_estimation), a sub-optimal training for the classifier may lead to a biased estimator for the likelihood ratio. This could lead to the wrong conclusions for an experiment. Therefore it is crucial to perform diagnostic checks on the SBI model, to validate that the output is what we expect. We can do this using independent simulation samples, different from those used for training. Section 4 of the following paper by the ATLAS collaboration discusses some of these diagnostic checks in detail, and how we can calibrate the classifier to give good performance.

[2] - ATLAS Collaboration, *An implementation of neural simulation-based inference for parameter estimation in ATLAS*, Rept. Prog. Phys 88 (2025) 6, 067801. [arXiv:2412.01600](https://arxiv.org/abs/2412.01600)

## Inclusion of systematic uncertainties
All above sections assume that our simulators are "faithful" i.e. they provide an accurate description of the data under some hypothesis/parameter value. In practice, there are always differences between the observed data and the synthetic data which leads to systematic uncertainties in the measurements. In the standard inference procedure, we introduce so-called nuisance parameters to account for these systematic effects. These can be incorporated into an SBI treatment by conditioning the classifiers on the nuisance parameters, in addition to the parameters of interest.

This concept is discussed in Section 5 of [1]. Further background information can be found in the following papers:

[3] - K. Cranmer, J. Pavez and G. Louppe, *Approximating Likelihood Ratios with Calibrated Discriminative Classifiers* (2016). [arXiv:1506.02169](https://arxiv.org/abs/1506.02169)

[4] - A. Ghost, B. Nachman and D. Whiteson, *Uncertainty-aware machine learning for high energy physics*, Phys, Rev. D 104 (2021) 056026. [arXiv:21.05.08742](https://arxiv.org/abs/2105.08742)

## Learning the density directly
Using classifiers for SBI is just one approach, where we learn the likelihood-density-ratio. Modern ML techniques, including generative models, can be used to learn the probability density directly (i.e. not the ratio). One architecture, known as conditional normalising flows, map the observable space to some simple latent space (of the same dimension), conditional on the parameter values $\theta$. The trained model can then be used simultaneously for both generating new synthetic data (generative AI) and for inference. Some references are provided below on this topic.

[5] - G. Papamakarios et al, *Normalizing flows for probabilistic modeling and inference*, JMLR 22 (2021) 1-64 [Link](https://jmlr.org/papers/volume22/19-1028/19-1028.pdf)

## Bayesian approach: neural posterior estimation
This notebook presented SBI in a frequentist paradigm. Of course the idea extends to Bayesian inference! Many applications look at learning the posterior directly. The references below concern the use of SBI for Bayesian inference in cosmology and gravitational waves, respectively.

[6] - N. Jeffey & B. Wandelt, *Solving high-dimensional parameter inference: marginal posterior densities & moment networks*, NeurIPS (2020) [arXiv:2011.05991](https://arxiv.org/pdf/2011.05991)

[7] - M. Dax et al, *Real-time gravitational-wave science with neural posterior estimation*, PRL (2021) [arXiv:2106.12594](https://arxiv.org/abs/2106.12594)

</dev>

# 